# Initialization

In [1]:
!pip install boto3 progressbar2 sfdmap GPUtil

  Using cached boto3-1.23.0-py3-none-any.whl (132 kB)
  Using cached progressbar2-4.0.0-py2.py3-none-any.whl (26 kB)
  Using cached sfdmap-0.1.1-py3-none-any.whl
  Using cached GPUtil-1.4.0-py3-none-any.whl
  Using cached jmespath-1.0.0-py3-none-any.whl (23 kB)
  Using cached s3transfer-0.5.2-py3-none-any.whl (79 kB)
  Using cached botocore-1.26.0-py3-none-any.whl (8.7 MB)
  Using cached python_utils-3.2.2-py2.py3-none-any.whl (20 kB)


In [2]:
# imports
import pandas as pd
import numpy as np
import os
import sys
import pickle
from matplotlib import pyplot as plt
import matplotlib
import boto3

# random seed
seed = 42
np.random.seed(seed)

# local files paths
local_home_dir_path = os.path.expanduser("~")
local_work_dir_path = os.path.join(local_home_dir_path, 'git')
local_code_dir_path = os.path.join(local_work_dir_path , 'code')

# S3 file paths
endpoint_url = 'https://s3-west.nrp-nautilus.io'
bucket_name = 'tau-astro'
prefix = 'almogh'
s3_work_dir_path = os.path.join(prefix, 'workdir3')
s3_saves_dir_path = os.path.join(s3_work_dir_path , 'model_saves')
s3_data_dir_path = os.path.join(s3_work_dir_path , 'data')
s3_v2_data_ver_dir_path = os.path.join(s3_data_dir_path,'100K_V2')
s3_v4_data_ver_dir_path = os.path.join(s3_data_dir_path,'100K_V4')

s3_client = boto3.client("s3", endpoint_url=endpoint_url)

# adding code folder to path
sys.path.insert(1, local_code_dir_path)
from s3 import to_s3_npy, to_s3_pkl, from_s3_npy, from_s3_pkl, to_s3_fig

# Infer RF

## Loading data

In [3]:
# load data
print('Loading data and creating dataset')
gs = from_s3_pkl(s3_client, bucket_name, os.path.join(s3_v4_data_ver_dir_path,'gs_test_V4.pkl'))
X = from_s3_npy(s3_client, bucket_name, os.path.join(s3_v2_data_ver_dir_path, 'spec.npy'))
full_wl_grid = from_s3_npy(s3_client, bucket_name, os.path.join(s3_data_dir_path, 'wl_grid.npy'))
wl_grid = from_s3_npy(s3_client, bucket_name, os.path.join(s3_v4_data_ver_dir_path, 'wl_100K_V4.npy'))
start_i = (np.abs(full_wl_grid - wl_grid[0])).argmin()
end_i = 1+(np.abs(full_wl_grid - wl_grid[-1])).argmin()
X = X[gs.index, start_i:end_i]

Loading data and creating dataset
loading from uri: s3://tau-astro/almogh/workdir3/data/100K_V4/gs_test_V4.pkl
loading from uri: s3://tau-astro/almogh/workdir3/data/100K_V2/spec.npy
loading from uri: s3://tau-astro/almogh/workdir3/data/wl_grid.npy
loading from uri: s3://tau-astro/almogh/workdir3/data/100K_V4/wl_100K_V4.npy


In [4]:
assert not np.any(np.isnan(X)), 'NaN!'

## Load RF

In [7]:
load_RF_name = 'simple___2022_05_07___18_57_07___100K_V4_training_set'
s3_load_dir_path = os.path.join(s3_saves_dir_path, 'RF', load_RF_name)
print('loading from folder (S3): {0}'.format(s3_load_dir_path))

from CustomRandomForest import CustomRandomForest
rf = CustomRandomForest.load_s3(s3_client, bucket_name, os.path.join(s3_load_dir_path, 'crf.pkl'))

loading from folder (S3): almogh/workdir3/model_saves/RF/simple___2022_05_07___18_57_07___100K_V4_training_set
loading from uri: s3://tau-astro/almogh/workdir3/model_saves/RF/simple___2022_05_07___18_57_07___100K_V4_training_set/crf.pkl


## Predict on test set

In [8]:
print('Applying the RF (calculate leaves)')
X_leaves = rf.apply(X)

print('Predicting fully')
Y_hat = rf.predict_full_from_leaves(X_leaves)

print('Calculating the similarity matrix')
from CustomRandomForest import build_similarity_matrix
sim_mat = build_similarity_matrix(X_leaves, Y_hat)

Applying the RF (calculate leaves)
apply: starting 500 jobs


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    4.9s


Predicting fully
Calculating the similarity matrix


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    5.8s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 616 tasks      | elapsed:   14.1s
[Parallel(n_jobs=-1)]: Done 850 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 1120 tasks      | elapsed:   24.1s
[Parallel(n_jobs

assembling the matrix.


In [12]:
print('Calculating the distance matrix and weirdness scores')
dist_mat_hat_test_set = 1 - sim_mat
weird_scores_hat_test_set = np.mean(dist_mat_hat_test_set, axis=1)

Calculating the distance matrix and weirdness scores


# save

In [13]:
print('Saving the weirdness scores')
to_s3_npy(weird_scores_hat_test_set, s3_client, bucket_name, os.path.join(s3_load_dir_path, 'weird_scores_hat_test_set.npy'))

print('Saving the dissimilarity matrix')
to_s3_npy(dist_mat_hat_test_set, s3_client, bucket_name, os.path.join(s3_load_dir_path, 'dist_mat_hat_test_set.npy'))

Saving the weirdness scores
saving to uri: s3://tau-astro/almogh/workdir3/model_saves/RF/simple___2022_05_07___18_57_07___100K_V4_training_set/weird_scores_hat_test_set.npy
Saving the dissimilarity matrix
saving to uri: s3://tau-astro/almogh/workdir3/model_saves/RF/simple___2022_05_07___18_57_07___100K_V4_training_set/dist_mat_hat_test_set.npy


True